# Send REST requests to the WeatherApp.API running in Aspire

In [1]:
// First set the base url.
var baseUrl = "https://localhost:7220";

## Healthcheck 💉

In [ ]:
#!set --value @csharp:baseUrl --name baseUrl
GET {{baseUrl}}/

## GET Weather Forecast 🌦️🌈🔆

In [ ]:
#!set --value @csharp:baseUrl --name baseUrl
GET {{baseUrl}}/v1/weather-forecast/Bristol/{{$datetime iso8601}}

## POST Colected Weather Data 🌡️☔💨

In [4]:
// Generate a random reference...
string[] fruits = ["apple", "banana", "cherry", "date", "elderberry", "fig", "grape", "honeydew", "kiwi", "lemon", "mango", "nectarine", "orange", "papaya", "quince", "raspberry", "strawberry", "tangerine", "watermelon", "guava"];
var randomReference = $"{fruits[new Random().Next(fruits.Length)]}-{Random.Shared.Next(10, 99)}";
randomReference.Display();

cherry-28

In [5]:
#!set --value @csharp:baseUrl --name baseUrl
#!set --value @csharp:randomReference --name randomReference

POST {{baseUrl}}/v1/collected-weather-data/Bristol/{{randomReference}}
Content-Type: application/json
x-request-id: {{$guid}}
    
{
    "points": [
        {
            "time": "{{$datetime iso8601}}",
            "windSpeedInMetersPerSecond": {{$randomInt 0 69}}.{{$randomInt 0 99}},
            "windDirection": "N",
            "temperatureReadingInDegreesCelcius": {{$randomInt -15 40}}.{{$randomInt 0 99}},
            "humidityReadingInPercent": {{$randomInt 20 100}}
        }
    ]
}

Name,Value
X-Request-ID,afaaecce-09a1-42dc-9f5a-c42d1ae3137a
traceparent,00-dbcba72f4b29fe39335e15401ac47f1c-a04c6e250926d5e3-00
Content-Type,application/json
Content-Length,279
Name,Value
Date,"Tue, 09 Sep 2025 18:16:43 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,application/json; charset=utf-8
Content-Length,76


<div style="display:block; height:10rem;"></div>

## GET Traces 🤖

In [ ]:
// Get traces and format nicely
using System.Net.Http;
using System.Text.Json;

public class TraceData
{
    public string Resource { get; set; } = string.Empty;
    public string Source { get; set; } = string.Empty;
    public string DisplayName { get; set; } = string.Empty;
    public string TraceId { get; set; } = string.Empty;
    public string SpanId { get; set; } = string.Empty;    
    public Dictionary<string, object> Tags { get; set; } = new();

    public override string ToString()
    {
        var thirdPart = string.Empty;
        if(DisplayName == "Domain Event Insertion")
            thirdPart = Tags["domain-event.eventclassName"].ToString().Split('.').LastOrDefault();

        if(DisplayName == "User Notification Sent")
            thirdPart = $"✉️ reference: {Tags["user-notification-event.reference"].ToString()}";

        if(DisplayName == "Outbox Item Insertion")
            thirdPart = $"📫 outbox id: {Tags["outbox-item.Id"].ToString()}";
                   
        return $"{Resource.PadRight(30, '-')}> {DisplayName.PadRight(30, '-')}> {thirdPart}";
    } 
        
}

var client = new HttpClient() { BaseAddress = new Uri("http://localhost:8000"), DefaultRequestHeaders = { { "X-Api-Key", "11119999" } } };
var response = await client.GetAsync("traces");
var responseJson = await response.Content.ReadAsStringAsync();
JsonSerializer.Deserialize<TraceData[]>(responseJson, JsonSerializerOptions.Web ).Display();